In [52]:
from config import RAW_DATA, PREPARE_DATA, AREA
import os
import json
from matplotlib import pyplot as plt

# TEST
print(RAW_DATA)
print(PREPARE_DATA)
print(type(AREA))


/Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/data
/Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare
<class 'list'>


In [53]:
# Clear column, calculator Power for PCSfile 
def clear_column(dataset):
    dataset.drop('PP_NAME', axis=1, inplace=True)
    dataset.drop('CC_NO', axis=1, inplace=True)
    dataset.drop('PCS_ID', axis=1, inplace=True)
    dataset.drop('Metering.TotWhOut', axis=1, inplace=True)
    dataset.drop('Operation.GriSwCnt', axis=1, inplace=True)
    dataset.drop('Metering.TotOpTms', axis=1, inplace=True)
    dataset.drop('Metering.TotFeedTms', axis=1, inplace=True)
    dataset.drop('Metering.GridMs.TotWhOut', axis=1, inplace=True)
    dataset.drop('GridMs.TotW', axis=1, inplace=True)
    dataset.drop('GridMs.Hz', axis=1, inplace=True)
    dataset.drop('Isolation.FltA', axis=1, inplace=True)
    dataset.drop('Isolation.LeakRis', axis=1, inplace=True)
    dataset.drop('DcMs.Vol[A]', axis=1, inplace=True)
    dataset.drop('DcMs.Vol[B]', axis=1, inplace=True)
    dataset.drop('DcMs.Amp[A]', axis=1, inplace=True)
    dataset.drop('DcMs.Amp[B]', axis=1, inplace=True)
    dataset.drop('DcMs.Amp[A1]', axis=1, inplace=True)
    dataset.drop('GridMs.PhV.phsA2B', axis=1, inplace=True)
    dataset.drop('GridMs.PhV.phsB2C', axis=1, inplace=True)
    dataset.drop('GridMs.PhV.phsC2A', axis=1, inplace=True)
    dataset.drop('GridMs.A.phsA', axis=1, inplace=True)
    dataset.drop('GridMs.A.phsB', axis=1, inplace=True)
    dataset.drop('GridMs.A.phsC', axis=1, inplace=True)
    dataset.drop('Operation.Health', axis=1, inplace=True)
    dataset.drop('Operation.Evt.Prio', axis=1, inplace=True)
    dataset.drop('Operation.Evt.Msg', axis=1, inplace=True)
    dataset.drop('Operation.Evt.Dsc', axis=1, inplace=True)
    dataset.columns = ['Timestamp','Watt[A]', 'Watt[B]']
    dataset['Watt[A]'].fillna(0, inplace=True)
    dataset['Watt[B]'].fillna(0, inplace=True)
    
    power = dataset["Watt[A]"] + dataset["Watt[B]"]
    dataset["power"] = power
    
    dataset.drop('Watt[A]', axis=1, inplace=True)
    dataset.drop('Watt[B]', axis=1, inplace=True)
    # pcs file
    dataset.columns = ['Timestamp','power']
    return dataset
        

In [54]:
# Get data from 2020 -> now for PCS
# Gộp data của từng PCS từ năm 2020 cho đến bây giờ , sau đó sẽ thu được dữ liệu thô cho từng PCS
import glob
import pandas as pd
from datetime import date
import os

#combine data pcs file 
def combine_data_pcs( plant_name, pcs_name, year, month):
    # get today
    today = date.today()
    today_year = today.year 
    today_month = today.month
    today_day = today.day

    # paths pcs_name file 
    paths = []
    while(1):
        m = str(month).zfill(2)
        max_d = 31 if month == 1 or month == 3 or month == 5 or month == 8 or month == 7 or month == 10 or month == 12 else 30
        if year == today_year and month == today_month:
            max_d = today_day
        elif month == 2:
            max_d = 28
        for i in range(max_d):
            d = str(i + 1).zfill(2)
            path =  RAW_DATA + '/'+ plant_name + '/cc/csv/' + str(year) + '/' + m + '/' + str(year) + m + d + '/' + pcs_name
            paths.append(path)
        month +=1
        if month == 13:
            month = 1
            year +=1
        if month > today_month and year == today_year:
            break
    # clear path pcs file without data
    clearly_paths_pcs = []
    for path in paths:
        try:
            dataset = pd.read_csv(path)
            if(len(dataset)>0):
                clearly_paths_pcs.append(path)
        except:
            continue
    combined_csv = pd.concat([pd.read_csv(f) for f in clearly_paths_pcs ])
    # clear column in combined_csv 
    combined_csv = clear_column(combined_csv)
    return combined_csv
  


In [55]:
# Lấy liệu về nhiệt lượng mặt trời từ năm 2020 cho đến bây giờ để ghép vào các pcs 
import glob
import pandas as pd
from datetime import date
import os

#combine data pcs file 
def combine_total_file(plant_name, year, month):
    
    today = date.today()
    today_year = today.year 
    today_month = today.month
    today_day = today.day

    # paths 
    total_paths = []
    
    for j in range(100):
        m = str(month).zfill(2)
        max_d = 31 if month == 1 or month == 3 or month == 5 or month == 8 or month == 7 or month == 10 or month == 12 else 30
        if year == today_year and month == today_month:
            max_d = today_day
        elif month == 2:
            max_d = 28
        for i in range(max_d):
            d = str(i + 1).zfill(2)
            path_total =  RAW_DATA + '/'+ plant_name +'/cc/csv/' + str(year) + '/' + m + '/' + str(year) + m + d + '/' + 'Total.csv'
            total_paths.append(path_total)
            
        month +=1
        if month == 13:
            month = 1
            year +=1
        if month > today_month and year == today_year:
            break
    clearly_paths_total = []
    # clear path total file without data
    for path in total_paths:
        try:
            dataset = pd.read_csv(path)
            if len(dataset)>0:
                clearly_paths_total.append(path)
        except:
            continue
    new_dataset = pd.DataFrame()
    if len(clearly_paths_total)>0:
        print(f'Start export combine total data {plant_name} \n...')
        total_csv = pd.concat([pd.read_csv(f) for f in clearly_paths_total])
        #clear column in total_csv
        new_dataset['Timestamp'] = total_csv['Timestamp']
        new_dataset['ExInsol'] = total_csv['Env.ExInsol']
        print('done')
        return new_dataset
    else:
        print('done')
        return new_dataset


In [56]:
from datetime import datetime

# Dùng cho dữ liệu của khu vực hokuto_soeyama
def parse_time1(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S JST')

# Dùng cho khu vực bình thường 
def parse_time2(x):
    return datetime.strptime(x, '%Y/%m/%d %H:%M JST')
    
    

In [57]:
# Gép lại dữa dữ liệu điện lượng từ pcs và dữ liệu mặt trời từ total 
def prepare_data(plant_name, list_pcs):
    # dataset solar
    dataset_total = combine_total_file(plant_name, 2018, 1)
    
    print(f'Start run prepare data pcs in {plant_name}')
    list_combined_pcs_paths = []
    data_paths = []
    for pcs_name in list_pcs:
        print(f'Start export {pcs_name} \n...')
        dataset = combine_data_pcs(plant_name, pcs_name, 2020, 1)
        # merge with dataset solar
        dataset = dataset.merge(dataset_total, how='inner', on='Timestamp')
        # prepare data 
        if plant_name == 'hokuto_soeyama':
            dataset['Timestamp'] = [parse_time1(x) for x in dataset['Timestamp']]
        else:
            dataset['Timestamp'] = [parse_time2(x) for x in dataset['Timestamp']]
            
        dataset.index = pd.to_datetime(dataset['Timestamp'])
        dataset["month"] = dataset['Timestamp'].index.month
        dataset["day"] = dataset['Timestamp'].index.day
        dataset["hour"] = dataset['Timestamp'].index.hour
        dataset["min"] = dataset['Timestamp'].index.minute
        dataset.drop('Timestamp', axis=1, inplace=True)
        dataset.columns = ['Power', 'Solar', 'Month', 'Day', 'Hour', 'Min']
        
        # 1 hour get 00 min and  30  min
        dataset = dataset[dataset['Min'] % 30 == 0]
        # export csv
        path_folder =  PREPARE_DATA + '/'+ plant_name +'/'
        os.makedirs(path_folder, exist_ok=True)
        path_file = path_folder + pcs_name
        
        dataset.to_csv(path_file, index=True, encoding='utf-8-sig')
        print(f'Export prepare file to {path_file}')
    print(f'...............\nDone {plant_name}')
    

    

In [58]:
# prepare_data 
# data_paths = prepare_data(plant_name)

In [59]:
# Lấy mốc 1/12/2021 để lấy danh sách các pcs của từng khu vực
import os 
from datetime import date
def get_list_pcs(plant_name):
    today = date.today()
    year = today.year 
    
    # Hiện tại Hokuto Soeyama chỉ có dữ liệu đến năm 2020 nên xét path = 2020 
    if plant_name == 'hokuto_soeyama':
        path = RAW_DATA + '/'+ plant_name + '/cc/csv/' + str(2020) + '/' + '01' + '/' + str(2020) + '01' + '01'
    # Hiện tại hyuga đang để chia dữ liệu thành 2 phần : không theo mẫu /cc/csv/ mà chia là /cc1/csv và /cc2/csv 
    elif plant_name == 'hyuga':
        path1 = RAW_DATA + '/'+ plant_name + '/cc1/csv/' + str(year) + '/' + '01' + '/' + str(2020) + '01' + '01'
        path2 = RAW_DATA + '/'+ plant_name + '/cc2/csv/' + str(year) + '/' + '01' + '/' + str(2020) + '01' + '01'
    elif plant_name == 'kenebetsu':
        path = RAW_DATA + '/'+ plant_name + '/cc/csv/' + str(year) + '/' + '01' + '/' + str(year) + '01' + '01'
    else:
        path = RAW_DATA + '/'+ plant_name + '/cc/csv/' + str(year) + '/' + '01' + '/' + str(year) + '01' + '12'
    
    try:
        if plant_name == 'hyuga':
            list_dir = os.listdir(path1)
            for file in os.listdir(path2):
                if file != 'Total.csv' and file != 'TotalAbb.csv':
                    list_dir.append(file)
                    
            # xoá file Total.csv và TotalAbb.csv 
            list_dir.remove('Total.csv')
            list_dir.remove('TotalAbb.csv')
        else:
            list_dir = os.listdir(path)
            list_dir.remove('Total.csv')
            list_dir.remove('TotalAbb.csv')
            
        return list_dir
    except:
        return []
# get_list_pcs('morai')

In [72]:
# Lấy danh sách các vùng với các PCS, Hiện tại chấp nhận bỏ các vùng data lỗi hoặc khác format 
dict_plants = {}
max = 0
for name_area in AREA:
    dict_plants[name_area] = get_list_pcs(name_area)
    print(f'==================================== {name_area} ====================================')
    print(dict_plants[name_area])
    max += len(dict_plants[name_area])

print(max)
    

==================================== hobetsu ====================================
['PCS#1900826152.csv', 'PCS#1900844891.csv', 'PCS#1900819406.csv', 'PCS#1900819374.csv', 'PCS#1900819376.csv', 'PCS#1900819438.csv', 'PCS#1900844887.csv', 'PCS#1900826197.csv', 'PCS#1900844882.csv', 'PCS#1900821638.csv', 'PCS#1900819458.csv', 'PCS#1900844953.csv', 'PCS#1900821715.csv', 'PCS#1900819506.csv', 'PCS#1900819466.csv', 'PCS#1900819499.csv', 'PCS#1900844995.csv', 'PCS#1900821710.csv', 'PCS#1900819891.csv', 'PCS#1900844969.csv', 'PCS#1900819461.csv', 'PCS#1900821713.csv', 'PCS#1900819500.csv', 'PCS#1900821699.csv', 'PCS#1900844997.csv', 'PCS#1900821680.csv', 'PCS#1900821695.csv', 'PCS#1900819862.csv', 'PCS#1900821696.csv', 'PCS#1900844973.csv', 'PCS#1900821718.csv', 'PCS#1900819867.csv', 'PCS#1900819456.csv', 'PCS#1900819442.csv', 'PCS#1900819481.csv', 'PCS#1900821620.csv', 'PCS#1900821542.csv', 'PCS#1900819747.csv', 'PCS#1900821554.csv', 'PCS#1900837915.csv', 'PCS#1900821636.csv', 'PCS#1900819408

In [61]:
# Chuẩn bị data cho tất cả các plant 
for plant_name, list_pcs in dict_plants.items():
    prepare_data(plant_name, list_pcs)
print('=======================================================================')
print('Done prepare all areas ')
    

Start export combine total data hobetsu 
...
done
Start run prepare data pcs in hobetsu
Start export PCS#1900826152.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hobetsu/PCS#1900826152.csv
Start export PCS#1900844891.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hobetsu/PCS#1900844891.csv
Start export PCS#1900819406.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hobetsu/PCS#1900819406.csv
Start export PCS#1900819374.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hobetsu/PCS#1900819374.csv
Start export PCS#1900819376.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hobetsu/PCS#1900819376.csv
Start export PCS#

Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hobetsu/PCS#1900821552.csv
Start export PCS#1900821624.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hobetsu/PCS#1900821624.csv
...............
Done hobetsu
Start export combine total data hokkaido_base_kita4sen 
...
done
Start run prepare data pcs in hokkaido_base_kita4sen
Start export PCS#1900876360.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hokkaido_base_kita4sen/PCS#1900876360.csv
Start export PCS#1900872528.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hokkaido_base_kita4sen/PCS#1900872528.csv
Start export PCS#1900876207.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/

Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hokuto_soeyama/PCS#1900826136.csv
Start export PCS#1900845097.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hokuto_soeyama/PCS#1900845097.csv
Start export PCS#1900838716.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hokuto_soeyama/PCS#1900838716.csv
Start export PCS#1900826311.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hokuto_soeyama/PCS#1900826311.csv
Start export PCS#1900845675.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/hokuto_soeyama/PCS#1900845675.csv
Start export PCS#1900845099.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_

Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/kakenouma/PCS#1901255868.csv
Start export PCS#1901255921.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/kakenouma/PCS#1901255921.csv
Start export PCS#1901256431.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/kakenouma/PCS#1901256431.csv
Start export PCS#1901256394.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/kakenouma/PCS#1901256394.csv
Start export PCS#1901256586.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook/work/model/prepare/kakenouma/PCS#1901256586.csv
Start export PCS#1901256960.csv 
...
Export prepare file to /Users/nguyenminhdan/afterFIT/monitoring/sunny_mon/services/mon-notebook

In [62]:
# Chia data ra de train 
from sklearn import tree
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

def decision_tree(df_train, df_test):
    predictors = ["Solar", "Month", "Day", "Hour"]
    target = "Power"

    train_X = df_train[predictors]
    train_y = df_train[target]

    # 検証用データ
    test_X = df_test[predictors]
    test_y = df_test[target]

    model = tree.DecisionTreeRegressor()
    model.fit(train_X, train_y)
    print(f"R^2:{model.score(test_X,test_y)}")
    
    return model.score(test_X,test_y)



In [70]:
# train data with all pcs in plant
import pandas as pd

def excute(dict_plants):
    # path data for pcs 
#     df = pd.DataFrame({'PP_Name':[], 'PCS_ID':[], 'R2':[]})
    columns1 = []
    columns2 = []
    columns3 = []
    
    print('============ Trainning by Decision tree ============')
    for plant_name, list_pcs in dict_plants.items():
        print('****************************')
        print(f'{plant_name}')
        for pcs_name in list_pcs:
            columns1.append(plant_name)
            columns2.append(pcs_name)
            path = PREPARE_DATA + '/'+ plant_name +'/' + pcs_name
            df = pd.read_csv(path, header=0, index_col=0)
            
            # chia đôi lượng dữ liệu 
            n_train_hours = int(len(df)/2)
            
            df_train = df.iloc[:n_train_hours,].dropna(axis=1,how="all").dropna() # Nanしか入っていない列を削除し、PowerVolumeがNanの行を削除
            df_test = df.iloc[n_train_hours:,].dropna(axis=1,how="all").dropna()
            print(f'{pcs_name}')
            
            columns3.append(decision_tree(df_train, df_test))
            
    df = pd.DataFrame({'PP_Name':columns1, 'PCS_ID': columns2, 'R2':columns3})
    df.to_csv('Results.csv')
    print('============ DONE ============')
    


In [71]:
# pred
excute(dict_plants)

============ Trainning by Decision tree ============
****************************
hobetsu
PCS#1900826152.csv
R^2:0.7218679745000354
PCS#1900844891.csv
R^2:0.7245008636792671
PCS#1900819406.csv
R^2:0.7561996184166122
PCS#1900819374.csv
R^2:0.7870742833221356
PCS#1900819376.csv
R^2:0.7426146773912117
PCS#1900819438.csv
R^2:0.7419112245517663
PCS#1900844887.csv
R^2:0.7399083404565605
PCS#1900826197.csv
R^2:0.7563511091761551
PCS#1900844882.csv
R^2:0.7238059701431468
PCS#1900821638.csv
R^2:0.7439307068253734
PCS#1900819458.csv
R^2:0.8120104480508048
PCS#1900844953.csv
R^2:0.7251546507207547
PCS#1900821715.csv
R^2:0.7846460603087297
PCS#1900819506.csv
R^2:0.797555801327468
PCS#1900819466.csv
R^2:0.7834068965979575
PCS#1900819499.csv
R^2:0.7356009710386755
PCS#1900844995.csv
R^2:0.7265756613452927
PCS#1900821710.csv
R^2:0.7972349564583555
PCS#1900819891.csv
R^2:0.7583721634732578
PCS#1900844969.csv
R^2:0.7225502466452106
PCS#1900819461.csv
R^2:0.7416093298186615
PCS#1900821713.csv
R^2:0.8216